In [16]:
# Dependencies and Setup
import pandas as pd
pd.set_option('mode.chained_assignment', None)

# File to Load
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [17]:
#preview dataframe
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [18]:
#display the total number of players
total_players = pd.DataFrame({"Total Players" : [purchase_data['SN'].nunique()]})
total_players

,Total Players
0,576


In [19]:
#find the number of unique items
unique_items = purchase_data["Item ID"].nunique()

#find the average price of the items and format
average_price = purchase_data["Price"].mean()
average_price = "${:.2f}".format(average_price)

#find the number of unique purchases
number_purchases = purchase_data["Purchase ID"].nunique()

#find the total revenue and format
total_revenue = purchase_data["Price"].sum()
total_revenue = "${:,.2f}".format(total_revenue)

#create a summary dataframe and display
summary_df = pd.DataFrame({"Number of Unique Items" : [unique_items], "Average Price" : [average_price],
                           "Number of Purchases" : [number_purchases], "Total Revenue" : [total_revenue]})
summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [20]:
#drop repeat players
unique_players = purchase_data.drop_duplicates(subset=['SN'])

#find the count of each player type
male_players = unique_players["Gender"].loc[unique_players["Gender"] == "Male"].count()
female_players = unique_players["Gender"].loc[unique_players["Gender"] == "Female"].count()
sum_players = unique_players['SN'].nunique()
other_players = sum_players - (male_players + female_players)

#calculate the percentages of each player type and format
pc_male = (male_players/sum_players)*100
pc_male = "{:,.2f}%".format(pc_male)
pc_female = (female_players/sum_players)*100
pc_female = "{:,.2f}%".format(pc_female)
pc_other = (other_players/sum_players)*100
pc_other = "{:,.2f}%".format(pc_other)

#create a summary dataframe and display
genderdemo_df = pd.DataFrame({"Gender" : ["Male", "Female", "Other/Non-Disclosed"], 
                              "Total Count" : [male_players, female_players, other_players],
                              "Percentage of Players" : [pc_male, pc_female, pc_other]})
genderdemo_df.set_index(["Gender"], inplace=True)
genderdemo_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%


In [21]:
#find the purchase count for each type of player
male_pur = purchase_data["Purchase ID"].loc[purchase_data["Gender"] == "Male"].count()
female_pur = purchase_data["Purchase ID"].loc[purchase_data["Gender"] == "Female"].count()
sum_pur = purchase_data["Purchase ID"].nunique()
other_pur = sum_pur - (male_pur + female_pur)

#find the average purchase price for each type of player and format
mavg_pur = (purchase_data["Price"].loc[purchase_data["Gender"] == "Male"].sum())/(male_pur)
mavg_pur = "${:.2f}".format(mavg_pur)
favg_pur = (purchase_data["Price"].loc[purchase_data["Gender"] == "Female"].sum())/(female_pur)
favg_pur = "${:.2f}".format(favg_pur)
oavg_pur = (purchase_data["Price"].loc[(purchase_data["Gender"] != "Male") & (purchase_data["Gender"] != "Female")].sum())/(other_pur)
oavg_pur = "${:.2f}".format(oavg_pur)

#find the total purchase value for each type of player
m_pt = (purchase_data["Price"].loc[purchase_data["Gender"] == "Male"].sum())
f_pt = (purchase_data["Price"].loc[purchase_data["Gender"] == "Female"].sum())
o_pt = (purchase_data["Price"].loc[(purchase_data["Gender"] != "Male") & (purchase_data["Gender"] != "Female")].sum())

#find the average total purchase per person 
m_app = m_pt / male_players
f_app = f_pt / female_players
o_app = o_pt / other_players

#format the values
m_pt = "${:,.2f}".format(m_pt)
f_pt = "${:,.2f}".format(f_pt)
o_pt = "${:,.2f}".format(o_pt)
m_app = "${:.2f}".format(m_app)
f_app = "${:.2f}".format(f_app)
o_app = "${:.2f}".format(o_app)

#create dataframe and display
genderpur_df = pd.DataFrame({"Gender" : ["Male", "Female", "Other/Non-Disclosed"], 
                                   "Purchase Count" : [male_pur, female_pur, other_pur],
                                   "Average Purchase Price" : [mavg_pur, favg_pur, oavg_pur],
                                    "Total Purchase Value" : [m_pt, f_pt, o_pt],
                                    "Average Total Purchase Per Person" : [m_app, f_app, o_app]})
genderpur_df.set_index(["Gender"], inplace=True)
genderpur_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
Gender,,,,
Male,652,$3.02,"$1,967.64",$4.07
Female,113,$3.20,$361.94,$4.47
Other/Non-Disclosed,15,$3.35,$50.19,$4.56


In [22]:
#create bins for each age group
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 1000]
bin_names = ["<10", "10-14", "15-19", "20-24", 
             "25-29", "30-34", "35-39", "40+"]

#add age group column to dataframe
unique_players["Age Group"] = pd.cut(unique_players["Age"], age_bins, labels=bin_names, include_lowest=True)

In [23]:
#make groupby
age_groups = unique_players.groupby("Age Group")

#get counts of each age group
age_count = age_groups["Purchase ID"].count()
age_count_df = age_count.to_frame()

#get percentage of players in each age group
age_total = age_count_df["Purchase ID"].sum()
age_pc = (age_count_df/age_total)*100

#make summary dataframe, format, and display
age_summary = pd.merge(age_count_df, age_pc, on="Age Group")
rename_age_summary = age_summary.rename(columns={"Purchase ID_x":"Total Count",
                           "Purchase ID_y":"Percentage of Players"})
rename_age_summary["Percentage of Players"] = rename_age_summary["Percentage of Players"].map('{:,.2f}%'.format)
rename_age_summary

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [24]:
#create bins for age groups
age_bins = [0, 9, 14, 19, 24, 29, 34, 39, 1000]
bin_names = ["<10", "10-14", "15-19", "20-24", 
             "25-29", "30-34", "35-39", "40+"]

#add age group column to dataframe
purchase_data["Age Group"] = pd.cut(purchase_data["Age"], age_bins, labels=bin_names, include_lowest=True)

In [25]:
#make groupby
age_groups = purchase_data.groupby("Age Group")

#find purchase counts
pur_count = age_groups["Purchase ID"].count()

#find purchase totals
pur_total = age_groups["Price"].sum()

#find average purchase price
avg_pur = pur_total/pur_count

#find average total purchase per person
avg_pp = pur_total/age_count

#set series to dataframes
pur_count = pur_count.to_frame()
pur_total = pur_total.to_frame()
avg_pur = avg_pur.to_frame()
avg_pp = avg_pp.to_frame()

#merge dataframes into single summary
age_psummary = pd.merge(pur_count, avg_pur, on="Age Group")
age_psummary = pd.merge(age_psummary, pur_total, on="Age Group")
age_psummary = pd.merge(age_psummary, avg_pp, on="Age Group")

#rename columns and format dataframe
rename_age_psummary = age_psummary.rename(columns={"Purchase ID":"Purchase Count",
                                                   "0_x":"Average Purchase Price",
                                                  "Price":"Total Purchase Value",
                                                  "0_y":"Average Total Purchase Per Person"})
rename_age_psummary["Average Purchase Price"] = rename_age_psummary["Average Purchase Price"].map('${:,.2f}'.format)
rename_age_psummary["Total Purchase Value"] = rename_age_psummary["Total Purchase Value"].map('${:,.2f}'.format)
rename_age_psummary["Average Total Purchase Per Person"] = rename_age_psummary["Average Total Purchase Per Person"].map('${:,.2f}'.format)
rename_age_psummary

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [26]:
#make groupby
top_spenders = purchase_data.groupby(["SN"])

#find purchase counts and total purchase value
ts_pcount = top_spenders[["Price"]].count()
ts_price = top_spenders[["SN", "Price"]].sum()

#merge dataframes above and rename columns
ts_summary = pd.merge(ts_pcount, ts_price, on="SN")
rename_ts_summary = ts_summary.rename(columns={"Price_x":"Purchase Count",
                                                "Price_y":"Total Purchase Value"})

#sort the chart by total purchase value to find top 5 and isolate top 5
sort_ts_sum = rename_ts_summary.sort_values(by="Total Purchase Value", ascending=False)
new_ts = sort_ts_sum.iloc[:5]

#calculate average purchase price and insert new column
avg_pp = new_ts["Total Purchase Value"]/new_ts["Purchase Count"]
new_ts["Average Purchase Price"] = avg_pp

#format values and display dataframe
new_ts["Average Purchase Price"] = new_ts["Average Purchase Price"].map('${:,.2f}'.format)
new_ts["Total Purchase Value"] = new_ts["Total Purchase Value"].map('${:,.2f}'.format)
new_ts

,Purchase Count,Total Purchase Value,Average Purchase Price
SN,,,
Lisosia93,5,$18.96,$3.79
Idastidru52,4,$15.45,$3.86
Chamjask73,3,$13.83,$4.61
Iral74,4,$13.62,$3.40
Iskadarya95,3,$13.10,$4.37


In [27]:
#make groupby
pop_items = purchase_data.groupby(["Item ID", "Item Name"])

#find item counts and total purchase value
item_count = pop_items[["Price"]].count()
item_price = pop_items[["Price"]].sum()

#merge dataframe above and rename columns
item_summary = pd.merge(item_count, item_price, on=["Item ID", "Item Name"])
rename_item_summary = item_summary.rename(columns={"Price_x":"Purchase Count",
                                                   "Price_y":"Total Purchase Value"})

#sort the chart by purchase count to find top 5 and isolate top 5
sort_item_sum = rename_item_summary.sort_values(by="Purchase Count", ascending=False)
new_item = sort_item_sum.iloc[:5]

#calculate average purchase price and insert new column
avg_price = new_item["Total Purchase Value"]/new_item["Purchase Count"]
new_item["Average Item Price"] = avg_price

#format values and display dataframe
new_item["Average Item Price"] = new_item["Average Item Price"].map('${:,.2f}'.format)
new_item["Total Purchase Value"] = new_item["Total Purchase Value"].map('${:,.2f}'.format)
new_item

,,Purchase Count,Total Purchase Value,Average Item Price
Item ID,Item Name,,,
92,Final Critic,13,$59.99,$4.61
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
145,Fiery Glass Crusader,9,$41.22,$4.58
132,Persuasion,9,$28.99,$3.22
108,"Extraction, Quickblade Of Trembling Hands",9,$31.77,$3.53


In [28]:
#sort the chart by total purchase value to find top 5 and isolate top 5
sort_item_tpv = rename_item_summary.sort_values(by="Total Purchase Value", ascending=False)
new_item = sort_item_tpv.iloc[:5]

#calculate average purchase price and insert new column
avg_price = new_item["Total Purchase Value"]/new_item["Purchase Count"]
new_item["Average Item Price"] = avg_price

#format values and display dataframe
new_item["Average Item Price"] = new_item["Average Item Price"].map('${:,.2f}'.format)
new_item["Total Purchase Value"] = new_item["Total Purchase Value"].map('${:,.2f}'.format)
new_item

,,Purchase Count,Total Purchase Value,Average Item Price
Item ID,Item Name,,,
92,Final Critic,13,$59.99,$4.61
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
82,Nirvana,9,$44.10,$4.90
145,Fiery Glass Crusader,9,$41.22,$4.58
103,Singed Scalpel,8,$34.80,$4.35


Three Observable Trends:
1. This game has an overwhelmingly male audience - 84% vs. 14% femal and 2% other/non-disclosed.
2. The majority of this game's players are 20-24 years old - nearly half of all players (45%) fall in this age group.
    a. This age group also has the most purchases and highest total purchase value.
3. Final Critic is the most popular item and the most profitable item. Oathbreaker is a close second in both categories.